In [2]:
from google.colab import files
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
import string
import gensim

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [42]:
folklore = pd.read_csv('08-folklore_deluxe_version.csv')
evermore = pd.read_csv('09-evermore_deluxe_version.csv')
lover = pd.read_csv('07-lover.csv')
reputation = pd.read_csv('06-reputation.csv')
the1989 = pd.read_csv('05-1989_deluxe.csv')
red = pd.read_csv('04-red_deluxe_edition.csv')
speak = pd.read_csv('03-speak_now_deluxe_package.csv')
fearless = pd.read_csv('02-fearless_taylors_version.csv')
taylor = pd.read_csv('01-taylor_swift.csv')

In [43]:
df = pd.concat([folklore, evermore, lover,reputation,the1989,red,speak,fearless,taylor], axis=0)
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,"I'm doing good, I'm on some new shit",1
1,folklore (deluxe version),the 1,1,"Been saying ""yes"" instead of ""no""",2
2,folklore (deluxe version),the 1,1,"I thought I saw you at the bus stop, I didn't ...",3
3,folklore (deluxe version),the 1,1,I hit the ground running each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [5]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-11-04 09:10:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-04 09:10:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-04 09:10:12--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [6]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
# preprocessing - zmienić didn't na did not itd
# usunac cyferki

In [44]:

translate_table = dict((ord(char), None) for char in string.punctuation) 
df['lyric'] = df['lyric'].apply(lambda x: x.translate(translate_table))

In [8]:
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,Im doing good Im on some new shit,1
1,folklore (deluxe version),the 1,1,Been saying yes instead of no,2
2,folklore (deluxe version),the 1,1,I thought I saw you at the bus stop I didnt th...,3
3,folklore (deluxe version),the 1,1,I hit the ground running each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [45]:
df['lyric'] = df['lyric'].apply(lambda x: x.lower())

In [46]:
def get_pos(word):

  #Słownik mapujący symbole części mowy zwrócone przez tagger do ich ekwiwalentów akceptowalnych przez lemmatizer
    tag_map = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  #Tagowanie części mowy i pobieranie jej symbolu
    tag = nltk.pos_tag([word])[0][1][0].upper()

  #Zwracanie zmapowanej wartości - w przypadku braku wartością domyślną jest rzeczownik
    return tag_map.get(tag, wordnet.NOUN)

In [47]:
lemmatizer = WordNetLemmatizer()

In [48]:
def lemmatize(text):
    return ' '.join(lemmatizer.lemmatize(word, get_pos(word)) for word in nltk.word_tokenize(text))

In [49]:
df['lyric'] = df['lyric'].apply(lambda x: x.replace("\u2005", " "))

In [50]:
df['lyric'] = df['lyric'].apply(lambda x: lemmatize(x))

In [51]:
all_lyrics = ' '.join(list(df['lyric']))

In [52]:
words = nltk.word_tokenize(all_lyrics)

words=[word.lower() for word in words]
vocab = set(words)

print(vocab)

{'pane', 'he—', 'sadness', 'mr', 'actually', 'off', 'tattoo', 'insult', 'gather', 'toy', 'trophy', 'home', 'onе', 'vine', 'betty', 'aim', 'hold', 'dom', 'infidelity', 'sister', 'god', 'thousand', 'awesome', 'wish', 'saltbox', 'laugh', 'bright', 'pastry', 'swear', 'photo', 'art', 'anybody', 'sleeve', 'faithless', 'blush', 'collect', 'applause', 'whod', 'be', 'aside', 'yesterday', 'helmet', 'fragile', 'locket', 'mmh', 'wax', 'beautiful', 'into', 'waitress', 'invite', 'conquest', 'anyone', 'follow', 'way', 'scarf', 'star', 'curse', 'fancy', 'broken', 'jetset', 'namedropping', 'backlogged', 'game', 'politics', 'pillow', 'aurora', 'eightynine', 'august', 'shouldve', 'sinkin', 'wild', 'ourselves', 'special', 'crashed', 'natural', 'sweatshirt', 'bustling', 'sat', 'only', 'health', 'claw', 'selfexpression', 'expect', 'pas', 'tore', 'ooh', 'mascara', 'touchin', 'confuse', 'jaguar', 'bond', 'overdramatic', 'iron', 'wrap', 'style', 'lifeline', 'goddamn', 'lobby', 'fair', 'want', 'suppose', 'blew'

In [53]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

In [54]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(vocab)

In [55]:
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1
      
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))
  
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
  
    return embedding_matrix_vocab

In [56]:
# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    './glove.6B.50d.txt', tokenizer.word_index,
  embedding_dim)
  
print("Dense vector for first word is => ",
      embedding_matrix_vocab[1])

Dense vector for first word is =>  [ 0.80574    -0.18509001  0.13801999  0.27680999  0.33407    -0.25659999
  0.46869999 -0.54062998  0.28737     0.27089     0.12679     0.15963
 -0.35407001  1.10829997  0.14695001 -0.43626001 -1.38569999 -1.04480004
  1.10769999 -0.67400998 -0.47310999 -0.53557003 -0.85367     0.080285
 -0.0082252   0.97487998 -0.21298     1.25199997  0.11226    -0.62677997
  0.47525001 -0.33473     0.29841     0.65394002 -0.74796999  0.63417
  0.85978001  0.22536001  0.73109001 -0.089299    1.00179994  0.39456001
 -0.37889999 -0.055091   -0.98641002  0.24167     1.03499997 -0.74792999
  0.68605    -0.34841999]


In [57]:
list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(16)]

'aim'

In [58]:
from sklearn.neighbors import NearestNeighbors

In [207]:
nn = NearestNeighbors(n_neighbors=4, algorithm='ball_tree', metric='euclidean')

In [208]:
nn.fit(embedding_matrix_vocab)

NearestNeighbors(algorithm='ball_tree', metric='euclidean', n_neighbors=4)

In [167]:
results = nn.kneighbors(embedding_matrix_vocab[[3]])

In [168]:
most_suitable = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(results[1][0][3])]

In [169]:
most_suitable

'disbelief'

In [179]:
list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(9)]

'gather'

In [ ]:
# 1 embedding_matrix_vocab to są embeddingi SŁÓWEK Z PIOSENEK
# 2 stworzyć dodatkowo embeddingi jakiś emocji i po nich odpytywać, zwraca się top 5 słów z jej piosenek wszystkich pasujacych do szukanego słowa
# 3 stworzyć tf idf i wybrać 5 top słów dla każdej piosenki
# 4 + dodatkowe features daceability itp ktore wybierze uzytkownik
# 4 porównać to, co wyszło w 2 + 4 z tym co wyszło w 3 + pobrane z api, znowu z knn

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
vectorizer = TfidfVectorizer(
    analyzer='word', 
    stop_words='english', 
    ngram_range = (1,1), 
    #max_df=0.5, 
    use_idf=True, 
    smooth_idf=True
    #max_features=1000
    )

In [33]:
df.head()

,album_name,track_title,track_n,lyric,line
0,folklore (deluxe version),the 1,1,Im do good Im on some new shit,1
1,folklore (deluxe version),the 1,1,Been say yes instead of no,2
2,folklore (deluxe version),the 1,1,I thought I saw you at the bus stop I didnt th...,3
3,folklore (deluxe version),the 1,1,I hit the ground run each night,4
4,folklore (deluxe version),the 1,1,I hit the Sunday matinée,5


In [68]:
grouped = df[['track_title', 'lyric']].groupby('track_title').agg(list).reset_index()

In [69]:
grouped['lyric'] = grouped['lyric'].apply(lambda x: ' '.join(x))

In [70]:
grouped.head()

,track_title,lyric
0,...Ready for It?,knew he be a killer first time that i saw him ...
1,22,it feel like a perfect night to dress up like ...
2,A Perfectly Good Heart,why would you wan na break a perfectly good he...
3,A Place In This World,i dont know what i want so dont ask me cause i...
4,Afterglow,i blew thing out of proportion now youre blue ...


In [71]:
tfidf_matrix  = vectorizer.fit_transform(grouped['lyric'].tolist() )

In [72]:
tfidf_features = sorted(vectorizer.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [73]:
df_tfidf = pd.DataFrame(data = tfidf_matrix.toarray(),index = range(len(grouped)),columns = tfidf_features)
df_tfidf

,158,16,16th,17,18,20,20year,22,230,2am,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.081395,0.036791,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.655537,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.046546,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.042452,0.020622,0.0,0.0
159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.057647,0.0,0.0


In [81]:
result = pd.concat([grouped, df_tfidf], axis=1, join="inner")

In [82]:
result.head()

,track_title,lyric,158,16,16th,17,18,20,20year,22,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,...Ready for It?,knew he be a killer first time that i saw him ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.081395,0.036791,0.000000,0.0,0.0
1,22,it feel like a perfect night to dress up like ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.655537,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,A Perfectly Good Heart,why would you wan na break a perfectly good he...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,A Place In This World,i dont know what i want so dont ask me cause i...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,Afterglow,i blew thing out of proportion now youre blue ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.046546,0.0,0.0


In [99]:
result.nlargest(3,'summer')

,track_title,lyric,158,16,16th,17,18,20,20year,22,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
21,Cruel Summer,yeah yeah yeah yeah fever dream high in the qu...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.029475,0.0,0.077662,0.0,0.0
139,​betty,betty i wont make assumption about why you swi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
135,seven,please picture me in the tree i hit my peak at...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


In [100]:
metadata = pd.read_csv('spotify_taylorswift.csv')

In [104]:
metadata = metadata[['name', 'danceability', 'acousticness', 'energy', 'liveness', 'valence']]

In [107]:
final = pd.merge(
    metadata,
    result,
    how="inner",
    on=None,
    left_on="name",
    right_on="track_title",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [108]:
final.head()

,name,danceability,acousticness,energy,liveness,valence,track_title,lyric,158,16,...,yesterday,yeugh,yogurt,york,youd,youll,young,youre,yourе,youve
0,22,0.661,0.00215,0.729,0.0477,0.668,22,it feel like a perfect night to dress up like ...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,A Perfectly Good Heart,0.483,0.00349,0.751,0.1280,0.268,A Perfectly Good Heart,why would you wan na break a perfectly good he...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,Afterglow,0.756,0.13000,0.449,0.1140,0.399,Afterglow,i blew thing out of proportion now youre blue ...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046546,0.0,0.000000
3,All Too Well,0.602,0.03820,0.609,0.1280,0.337,All Too Well,i walk through the door with you the air be co...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.098728
4,Bad Blood,0.646,0.08850,0.794,0.2010,0.287,Bad Blood,cause baby now weve get bad blood you know it ...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023505,0.0,0.117325


In [109]:
features = ['danceability', 'acousticness', 'energy', 'liveness', 'valence']

In [260]:
example = {'romance': 1}

In [261]:
# matrix for vocab: word_index
embedding_dim = 50
embedding = embedding_for_vocab(
    './glove.6B.50d.txt', example,
  embedding_dim)
  
print("Dense vector for first word is => ",
      embedding[1])

Dense vector for first word is =>  [ 2.59200007e-01  1.12230003e+00 -1.59300005e+00 -1.23690002e-01
 -1.50409997e-01  1.35230005e+00 -4.44849998e-01 -4.33490008e-01
 -6.17089987e-01  1.09710002e+00 -3.75299990e-01  7.91920006e-01
 -2.11909994e-01  3.76489997e-01  1.05530000e+00 -4.50250000e-01
 -4.01450008e-01  7.70549998e-02 -1.03349999e-01  4.41700011e-01
  4.38129991e-01  9.72060025e-01  9.50599983e-02  9.50380027e-01
  9.78630006e-01 -8.12189996e-01 -1.25880003e+00 -1.58629999e-01
  7.95669973e-01  1.23329997e-01  1.52730000e+00 -2.26129994e-01
  1.10670000e-01 -7.54309967e-02 -2.84299999e-02  9.93089983e-04
 -1.69369996e+00 -5.15169978e-01 -6.11320019e-01 -8.41170013e-01
  6.70820028e-02  1.83060005e-01  1.44930005e-01 -2.29589999e-01
 -7.61620030e-02  9.62060019e-02  1.29089999e+00 -5.79970002e-01
 -3.14269990e-01 -3.65330011e-01]


In [262]:
resultt = nn.kneighbors(embedding[[1]])

In [263]:
all_results = list(resultt[1][0])

In [265]:
top_words = [list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(x)] for x in all_results]

In [266]:
new_features = features + top_words

In [267]:
new_features

['danceability',
 'acousticness',
 'energy',
 'liveness',
 'valence',
 'romantic',
 'mystery',
 'revolves',
 'obsession']

In [269]:
final[new_features]

,danceability,acousticness,energy,liveness,valence,romantic,mystery,revolves,obsession
0,0.661,0.00215,0.729,0.0477,0.668,0.0,0.0,0.0,0.0
1,0.483,0.00349,0.751,0.1280,0.268,0.0,0.0,0.0,0.0
2,0.756,0.13000,0.449,0.1140,0.399,0.0,0.0,0.0,0.0
3,0.602,0.03820,0.609,0.1280,0.337,0.0,0.0,0.0,0.0
4,0.646,0.08850,0.794,0.2010,0.287,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
94,0.456,0.81400,0.265,0.0679,0.112,0.0,0.0,0.0,0.0
95,0.602,0.88800,0.494,0.0902,0.374,0.0,0.0,0.0,0.0
96,0.780,0.75000,0.361,0.1080,0.163,0.0,0.0,0.0,0.0
97,0.511,0.31200,0.478,0.1470,0.408,0.0,0.0,0.0,0.0


In [ ]:
# z tej reszty zrobic potem srednia, tak zeby to byl jeden feature
# knn do jakis dacneability itd a reszte do 1